# Chapter 5. 오차역전파법 (Backpropagation)  

* 수치 미분의 장점과 단점
    * 장점 - 단순하고 구현이 간단.
    * 단점 - 시간이 오래 걸림.
        * 가중치 매개변수의 기울기를 효율적으로 계산하기 위해 Backpropagation 도입.


* Backpropagation의 이해
    * 수식 - 설명 안함.
    * 계산 그래프(시각적) - CS231n 참고

## 5.1 계산 그래프 (Computational Graph)  

* 계산 그래프 - 계산 과정을 그래프로 표현. (TensorFlow와 비슷한 접근).
    * 노드(Node) - 원(O)으로 표현 - 연산
    * 에지(Edge) - 화살표 - 연산의 흐름

### 5.1.1 계산 그래프로 풀다  

*문제 1 : 현빈 군은 슈퍼에서 1개에 100원인 사과 2개를 샀습니다. 이때 지불금액을 구하세요. 단 소비세가 10% 부과됩니다.*  
(문제 1은 곱으로만 표현 가능)  

<center>**Figure 5-1 계산 그래프로 풀어본 문제 1의 답**</center>
![Figure_5-01](./images/Figure_5-01.png)  

**Figure 5-1**은 상수와 연산이 완전히 분리되어 있지 않으므로 상수와 연산자를 완전히 분리하면 **Figure 5-2**와 같이 표현됨.  
(이후에는 **Figure 5-2**와 같은 형식으로 표현)  

<center>**Figure 5-2 계산 그래프로 풀어본 문제 1의 답 : '사과의 개수'와 '소비세'를 변수로 취급해 원 밖에 표기**</center>
![Figure_5-02](./images/Figure_5-02.png)

*문제 2 : 현빈 군은 사과를 2개, 귤을 3개 샀습니다. 사과는 1개에 100원, 귤은 1개 150원입니다. 소비자세가 10%일 때 지불 금액을 구하세요.*  
(문제 2는 곱과 합으로 표현 가능)

<center>**Figure 5-3 계산 그래프로 풀어본 문제2의 답**</center>
![Figure_5-03](./images/Figure_5-03.png)

* 계산 그래프를 이용한 문제풀이의 흐름
    1. 계산 그래프를 구성한다.
    2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다.

2와 같이 왼쪽에서 오른쪽으로 진행하는 단계를 순전파 (Forward Propagation).  
반대로 오른쪽에서 왼쪽으로 진행하는 단계를 역전파 (Backpropagation).

### 5.1.2 국소적 계산  

계산 그래프의 특징은 **국소적 계산**을 전파함으로써 최종 결과를 얻는 것.  
**국소적이란** 자신과 직접 관계된 작은 범위.  
(전체와는 상관없이 자신과 관계된 정보만으로 다음 결과를 출력할 수 있음.)

<center>**Figure 5-4 사과 2개를 포함해 여러 식품을 구입하는 예**</center>  
![Figure_5-04](./images/Figure_5-04.png)

**Figure 5-4**에서 여러 식품을 구입하여 총금액이 4,000원.  
이후 사과를 추가로 더 구매한다면 두 값을 더하는 계산(4,000 + 200 -> 4,200)은 4,000이라는 숫자가 어떻게 계산되어 나왔느냐와 관계가 없음.  
결국 각 노드는 자신과 관련한 계산(Figure 5-4에서는 두 숫자의 덧셈) 외에는 신경쓸 사항이 없음.  


위에서 처럼 계산 그래프는 국소적 계산에만 집중하면 됨.  
**간단한 국소적인 계산의 조합으로 복잡한 계산 결과를 얻을 수 있음.**

### 5.1.3 왜 계산 그래프로 푸는가?  

* 계산 그래프의 이점
    * 국소적 계산
        * 각 노드의 단순한 계산으로 문제를 단순화 할 수 있음.
        * 각 노드의 중간 계산 결과를 모두 보관 가능.
    * **역전파를 통해 *미분*을 효율적으로 계산 가능.**


역전파 설명을 위해 문제 1로 다시 돌아감.  
*문제 1 : 현빈 군은 슈퍼에서 1개에 100원인 사과 2개를 샀습니다. 이때 지불금액을 구하세요. 단 소비세가 10% 부과됩니다.*  

*문제 1-1 : 사과 가격이 오르면 최종 금액에 어떤 영향을 끼치는지 알고 싶음.*  
위 문제는 **사과 가격에 대한 지불 금액의 미분**을 구하는 문제임.

사과 값을 $x$, 지불 금액을 $L$이라 했을 때 $\frac{\partial L}{\partial x}$을 구하는 문제.  
(사과 값이 **아주 조금** 올랐을 때 지불 금액이 얼마나 증가하느냐?)

**사과 가격에 대한 지불 금액의 미분**은 역전파를 하면 구할 수 있음.  
**Figure 5-5**는 계산 그래프 상의 역전파를 미분을 통혜서 구해 놓은 결과.(자세한 내용은 뒤에서)  

<center>**Figure 5-5 역전파에 의한 미분 값의 전달**</center>
![Figure_5-05](./images/Figure_5-05.png)

역전파는 순전파와는 반대 방향의 화살표(붉은 선)으로 표시.  
이 전파는 **국소적 미분**을 전달하고 그 미분 값을 화살표의 아래에 표시.  
이 예에서 역전파는 오른쪽에서 왼쪽으로 **1 -> 1.1 -> 2.2** 순으로 미분 값을 전달.  
이 결과로부터 **사과 가격에 대한 지불 금액의 미분**값은 2.2라는 것을 알 수 있음.  
사과가 1원 오르면 최종금액은 2.2원 오름.  
(사과 가격이 $x$만큼 오르면 최종금액은 2.2$x$만큼 오름.)  


## 5.2 연쇄법칙 (Chain Rule)  

**Figure 5-5**처럼 오른쪽에서 왼쪽으로 **국소적 미분**을 전달하는 원리는 **연쇄법칙(Chain Rule)**에 따른 것.  
이 절에서 연쇄법칙을 설명하고 그것이 그래프 상의 역전파와 같다는 사실을 확인.

### 5.2.1 계산 그래프의 역전파  

**Figure 5-6**에서 $y=f(x)$라는 계산의 역전파를 표현.  

<center>**Figure 5-6 계산 그래프의 역전파 : 순방향과는 반대 방향으로 국소적 미분을 곱한다.**</center>  
![Figure_5-06](./images/Figure_5-06.png)

**Figure 5-6**과 같이 역전파의 계산 절차는 신호 $E$에 노드의 국소적 미분$(\frac{\partial y}{\partial x})$를 곱한 후 다음 노드로 전달하는 것.  
1. **국소적 미분**은 순전파 때의  $y=f(x)$ 계산의 미분을 구한다는 것이며, 이는 $x$에 대한 $y$의 미분$(\frac{\partial y}{\partial x})$을 구한다는 뜻.  
(예 : $y=f(x)=x^2$이라면 $(\frac{\partial y}{\partial x}) = 2x$)
2. 그리고 이 국소적인 미분을 상류에서 전달된 값(이 예에서는 E)에 곱해 앞으로 전달하는 것.  

위의 방식을 따르면 목표로 하는 미분 값을 효율적으로 구할 수 있다는 것이 역전파의 핵심.

### 5.2.2 연쇄법칙이란?  

**합성 함수** - 여려 함수로 구성된 함수.  

예) $x = {(x + y)}^2$는 **Equation 5.1**처럼 두 개의 식으로 구성.  
$$ \mathbf{Equation \ 5.1} \\ z = t^2 \\  t = x + y $$

* 연쇄법칙
    * *합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.*  

**Equation 5.1**을 예로 설명, $\frac{\partial z}{\partial x}$ ($x$에 대한 $z$의 미분)은  $\frac{\partial z}{\partial t}$  ($t$에 대한 $z$의 미분)과  $\frac{\partial t}{\partial x}$($x$에 대한 $t$의 미분)의 곱으로 나타낼 수 있음.  
수식으로는 **Equation 5.2**처럼 표기.  

$$ \mathbf{Equation \ 5.2} \\ \frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} $$  

**Equation 5.2**는 다음과 같이 $\partial t$를 서로 지울 수 있음.

$$\frac{\partial z}{\partial x} = \frac{\partial z}{\cancel{\partial t}} \frac{\cancel{\partial t}}{\partial x}$$  

연쇄 법칙을 써서 **Equation 5.2**의 미분 $\frac{\partial z}{\partial x}$를 진행. 먼저 **Equation 5.1**의 국소적 미분(편미분)을 구함.  
$$ \mathbf{Equation \ 5.3} \\ \frac{\partial z}{\partial t} = 2t \\ \frac{\partial t}{\partial x} = 1 $$  

**Equation 5.3**과 같이 $\frac{\partial z}{\partial t}$는 $2t$이고, $\frac{\partial t}{\partial x}$는 1.(미분 공식에서 해석적으로 구한 결과)  
최종적으로 구하고자 하는 $\frac{\partial z}{\partial x}$는 **Equation 5.3**에서 구한 두 미분을 곱해서 계산.  

$$ \mathbf{Equation \ 5.4} \\ \frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = 2t \centerdot 1 = 2(x + y)$$  


<br><br><br>

$$
\begin{align*}
  & \phi(x,y) = \phi \left(\sum_{i=1}^n x_ie_i, \sum_{j=1}^n y_je_j \right)
  = \sum_{i=1}^n \sum_{j=1}^n x_i y_j \phi(e_i, e_j) = \\
  & (x_1, \ldots, x_n) \left( \begin{array}{ccc}
      \phi(e_1, e_1) & \cdots & \phi(e_1, e_n) \\
      \vdots & \ddots & \vdots \\
      \phi(e_n, e_1) & \cdots & \phi(e_n, e_n)
    \end{array} \right)
  \left( \begin{array}{c}
      y_1 \\
      \vdots \\
      y_n
    \end{array} \right)
\end{align*}
$$

### 5.2.3 연쇄법칙과 계산 그래프  

**Equation 5.4**의 연쇄법칙 계산을 계산 그래프로 표현하면 **Figure 5-7**처럼 표현 가능.  
(2제곱 계산은 '**$\ast\ast2$**'노드로 표현)  

<center>**Figure 5-7 *Equation 5.4*의 계산 그래프 : 순전파와는 반대 방향으로 국소적 미분을 곱하여 전달한다.**</center>
![Figure_5-07](./images/Figure_5-07.png)



<center>**Figure 5-8 계산 그래프의 역전파 결과에 따르면 $\frac{\partial z}{\partial x}$는 $2(x + y)$가 된다.**</center>
![Figure_5-08](./images/Figure_5-08.png)

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파  

<center>**Figure 5-9 덧셈 노드의 역전파 : 왼쪽이 순전파. 오른쪽이 역전파다. 덧셈 노드의 역전파는 입력 값을 그대로 흘려보낸다.**</center>
![Figure_5-09](./images/Figure_5-09.png)  

<center>**Figure 5-10 최종 출력으로 가는 계산의 중간에 덧셈 노드가 존재한다. 역전파에서는 국소적 미분이 가장 오른쪽의 출력에서  
시작하여 노드를 타고 역방향(왼쪽)으로 전파된다.**</center>
![Figure_5-10](./images/Figure_5-10.png)  

<center>**Figure 5-11 덧셈노드 역전파의 구체적인 예**</center>
![Figure_5-11](./images/Figure_5-11.png)  

### 5.3.2 곱셈 노드의 역전파  

<center>**Figure 5-12 곱셈 노드의 역전파 : 왼쪽이 순전파. 오른쪽이 역전파다.**</center>
![Figure_5-12](./images/Figure_5-12.png)

<center>**Figure 5-13 곱셈 노드 역전파의 구체적인 예**</center>
![Figure_5-13](./images/Figure_5-13.png)



### 5.3.3 사과 쇼핑의 예  

<center>**Figure 5-14 사과 쇼핑의 역전파 예**</center>
![Figure_5-14](./images/Figure_5-14.png)

<center>**Figure 5-15 사과와 귤 쇼핑의 역전파 예 : 빈 상자 안에 적절한 숫자를 넣어 역전파를 완성하시오.**</center>
![Figure_5-15](./images/Figure_5-15.png)

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층  

<center>**Figure 5-16 사과 2개 구입**</center>
![Figure_5-16](./images/Figure_5-16.png)



### 5.4.2 덧셈 계층  

<center>**Figure 5-17 사과 2개와 귤 3개 구입**</center>
![Figure_5-17](./images/Figure_5-17.png)



## 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층  

<center>**Figure 5-18 ReLU 계층의 계산 그래프**</center>
![Figure_5-18](./images/Figure_5-18.png)



### 5.5.2 Sigmoid 계층  

<center>**Figure 5-19 Sigmoid 계층의 계산 그래프(순전파)**</center>
![Figure_5-19](./images/Figure_5-19.png)  

#### 1단계  
![Figure_5-19-1](./images/Figure_5-19-1.png)  

#### 2단계  
![Figure_5-19-2](./images/Figure_5-19-2.png)  

#### 3단계  
![Figure_5-19-3](./images/Figure_5-19-3.png)  

#### 4단계  
<center>**Figure 5-20 Sigmoid 계층의 계산 그래프**</center>
![Figure_5-20](./images/Figure_5-20.png)  

<center>**Figure 5-21 Sigmoid 계층의 계산 그래프(간소화 버전)**</center>
![Figure_5-21](./images/Figure_5-21.png)  

<center>**Figure 5-22 Sigmoid 계층의 그래프 : 순전파의 출력 $y$만으로 역전파를 계산할 수 있다.**</center>
![Figure_5-22](./images/Figure_5-22.png)  



## 5.6 Affine/Softmax 계층 구현하기

### 5.6.1 Affine 계층  

<center>**Figure 5-23 행렬의 내적에서는 대응하는 차원의 원소 수를 일치시킨다.**</center>
![Figure_5-23](./images/Figure_5-23.png)  

<center>**Figure 5-24 Affine 계층의 계산 그래프 : 변수가 행렬임에 주의. 각 변수의 형상을 변수명 위에 표기했다.**</center>
![Figure_5-24](./images/Figure_5-24.png)  

<center>**Figure 5-25 Affine 계층의 역전파 : 변수가 다차원 배열임에 주의. 역전파에서의 변수 형상은 해당 변수명 아래에 표기했다.**</center>
![Figure_5-25](./images/Figure_5-25.png)  

<center>**Figure 5-26 행렬 내적('dot'노드)의 역전파는 행렬의 대응하는 차원의 원소 수가 일치하도록 내적을 조립하여 구할 수 있다.**</center>
![Figure_5-26](./images/Figure_5-26.png)  



### 5.6.2 배치용 Affine 계층  

<center>**Figure 5-27 배치용 Affine 계층의 계산 그래프**</center>
![Figure_5-27](./images/Figure_5-27.png)  

### 5.6.3 Softmax-with-Loss 계층  

<center>**Figure 5-28 입력 이미지가 Affine 계층과 ReLU 계층을 통과하며 변환되고, 마지막 Softmax 계층에 의해서 10개의 입력이 정규화된다.  
이 그림에서는 숫자 '0'의 점수는 5.3이며, 이것이 Softmax 계층에 의해서 0.008(0.8%)로 변환된다. 또, '2'의 점수는 10.1에서 0.991(99.1%)로 변환된다.**</center>
![Figure_5-28](./images/Figure_5-28.png)  

<center>**Figure 5-29 Softmax-with-Loss 계층의 계산 그래프**</center>
![Figure_5-29](./images/Figure_5-29.png)  

<center>**Figure 5-30 '간소화한' Softmax-with-Loss 계층의 계산 그래프**</center>
![Figure_5-30](./images/Figure_5-30.png)  

## 5.7 오차역전파법 구현하기

### 5.7.1 신경망 학습의 전체 그림

### 5.7.2 오차역전파법을 적용한 신경망 구현하기

### 5.7.3 오차역전파법으로 구한 기울기 검증하기

### 5.7.4 오차역전파법을 사용한 학습 구현하기